# Differential Evolution Neural Network training

Using differential evolution to train neural network. The task is basically the problem of optimizing the function of 83 arguments (number depends on architecture), which are the weights and biases of neural network. 

In [89]:
from keras.utils import to_categorical
from scipy import optimize as op
import pandas as pd
import numpy as np

In [90]:
# Upload the Iris dataset from file. 
dataset = pd.read_csv('iris.data', header=0)

# Shuffling dataset to split it into train and test set, and avoid high variance.
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset.head(10)

,sepal length,sepal width,petal length,petal width,class
0,5.6,2.8,4.9,2.0,Iris-virginica
1,6.1,3.0,4.9,1.8,Iris-virginica
2,5.7,2.6,3.5,1.0,Iris-versicolor
3,4.4,2.9,1.4,0.2,Iris-setosa
4,4.3,3.0,1.1,0.1,Iris-setosa
5,5.1,3.8,1.6,0.2,Iris-setosa
6,6.4,3.2,4.5,1.5,Iris-versicolor
7,5.6,3.0,4.5,1.5,Iris-versicolor
8,6.3,2.3,4.4,1.3,Iris-versicolor
9,5.7,2.9,4.2,1.3,Iris-versicolor


In [91]:
# Adding bias unit.
dataset.insert(0, 'bias', 1)

# Data standartization
for label in list(dataset)[1:-1]:
    dataset[label] = (dataset[label] - dataset[label].mean()) / dataset[label].abs().max()

# Mapping class names to numbers.
dataset['class'] = dataset['class'].map({'Iris-setosa':0, 
                                         'Iris-versicolor':1, 
                                         'Iris-virginica':2}).astype(int)
train = dataset[:100]
test = dataset[100:]

X = train.as_matrix(['bias',
                   'sepal length', 
                   'sepal width', 
                   'petal length', 
                   'petal width'])
y = train.as_matrix(['class'])
y = to_categorical(y)

X_test = test.as_matrix(['bias',
                   'sepal length', 
                   'sepal width', 
                   'petal length', 
                   'petal width'])
y_test = test.as_matrix(['class'])
y_test = to_categorical(y_test)
dataset.sample(10)

,bias,sepal length,sepal width,petal length,petal width,class
77,1,0.019831,-0.194091,0.179903,0.120533,2
146,1,-0.144726,0.033182,-0.356329,-0.399467,0
55,1,-0.005485,-0.080455,0.194396,0.280533,2
89,1,-0.182700,0.033182,-0.356329,-0.399467,0
25,1,0.171730,0.033182,0.324831,0.240533,2
127,1,0.057806,-0.035000,0.266860,0.240533,2
1,1,0.032489,-0.012273,0.165411,0.240533,2
125,1,0.083122,-0.057727,0.121932,0.120533,1
90,1,0.083122,-0.012273,0.295845,0.400533,2
137,1,0.121097,0.033182,0.310338,0.440533,2


In [115]:
# Defining Rectified Linear Unit, which is used as actiation function here.
def relu(x):
    return x * (x > 0)

# Defining softmax, which is used in the output layer.
def softmax(a):
    s = np.exp(a - np.max(a))
    if a.ndim <3: return s/s.sum(0) 

# Defining Function to minimize.
def f(theta):
    # Reshaping 1D input weights to 2 2D matrix with bias.    
    theta1 = theta[:40]
    bias1 = theta[40:50]
    theta2 = theta[50:80]
    bias2 = theta[80:]
    theta1 = theta1.reshape((10, 4))
    theta2 = theta2.reshape((3, 10))
    theta1 = np.column_stack((bias1, theta1))
    theta2 = np.column_stack((bias2, theta2))
    
    # Choosing a batch of 3 random samples.
    batch_idx = np.random.choice(X.shape[0], 1, replace=False)
    x_batch = X[batch_idx]
    y_batch = y[batch_idx]
    
    # Calculationg the loss via cross entropy.
    h = relu(theta1.dot(x_batch.T))
    h = np.row_stack((np.ones(h.shape[1]), h))
    out = softmax(theta2.dot(h))
    loss = -np.log((y_batch.T * out).sum(0))
    return loss.sum()

In [116]:
# bounds = [(-2., 2.)]*70
# op.differential_evolution(f, bounds, maxiter=50000)
bounds = [(-2., 2.)]*83
op.differential_evolution(f, bounds, maxiter=50000)

     fun: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 1099854
     nit: 881
 success: True
       x: array([ 0.94687211, -1.17460209,  1.31392388,  1.80457367,  0.1673272 ,
       -0.61202518, -1.43140639,  1.09407974, -1.32407377, -0.22533852,
        0.85725644, -0.65684114,  0.35252209, -0.54304552, -1.5368403 ,
        0.31183973, -0.88354785, -0.37191471,  1.05979869,  0.65025884,
       -0.33369859, -1.65704745,  1.24157308,  0.54328191,  0.61338311,
       -0.38718278,  0.38212946, -0.57253222,  0.59588837,  1.94174821,
        0.04292019,  1.95610101, -0.81583466,  1.24463668, -0.41745033,
        1.76090925,  1.77253165,  1.96020692, -0.11760402,  1.99500498,
        1.90604524, -0.13210263, -0.88899273, -0.9338501 ,  1.58071441,
        1.92337724, -0.04912468,  1.68260161,  1.50712985,  1.54080191,
       -0.99211549, -0.68746827,  0.34921015,  0.39645783, -1.00724935,
       -1.61396399,  0.12846123, -1.15706667, -0.67966803, -0.11938515,
       -0.41262